In [ ]:
import pickle
from collections import OrderedDict
import pandas as pd
from pandas import DataFrame
import pickle
import sqlite3
import datetime
from dateutil.parser import parse

In [ ]:
# sections = ['politics','economic','society','global','culture','digital']
# idate_list = ['2018-08-16','2018-08-17','2018-08-18','2018-08-19',
#               '2018-08-20','2018-08-21','2018-08-22','2018-08-23']

# section='economic'
# i_date = '2018-08-16'

sections=['politics','economic']
idate_list = ['2018-08-16','2018-08-17']

path_sec = 'db/inverted_index/'
path_idf = 'db/unique_nouns_idf/'
path_i = 'db/i_value/'

# a_table = DataFrame({'rank' : range(1,11)})
# a_table.set_index('rank',inplace=True)
# a_list = [0.8,0.81,0.82,0.83,0.84,0.85,0.86,0.87,0.88,0.89,0.9]
a = 0.825

In [ ]:
kw_total=DataFrame()
for section in sections:
    conn = sqlite3.connect(f'{path_i}test.db')
    cur = conn.cursor()

    try:
        cur.execute(f"CREATE TABLE I_value_{section} \
                                  (index      INTEGER PRIMARY KEY, \
                                   idate      DATE, \
                                   term       TEXT, \
                                   docsNum    INTEGER, \
                                   df_sec     REAL, \
                                   idf_all    REAL, \
                                   i_value    REAL, \
                                   rank_i     INTEGER, \
                                   FOREIGN KEY (term) REFERENCES U_Term(u_term))")
        conn.commit()
    except:
        pass

    def i_value_cal(a, df, idf):
        return (df * idf) / (a*df + (1-a)*idf)

    itb_idate=DataFrame()
    kw_idate=DataFrame()
    # for a in a_list:
    for idate in idate_list: 
        inverted = pd.read_pickle(f'{path_sec}inverted_index_{section}_{idate}.pkl')
        idf = pd.read_pickle(f'{path_idf}unique_nouns_idf.pkl')
        print(f'{idate} 키워드')

        term=[]
        docsNum=[]
        idf_all=[]
        itb=DataFrame()


        for t, docs in inverted.items():
            term.append(t)
            docsNum.append(len(docs))
            idf_all.append(idf[t])
        itb = DataFrame({'term':term, 'docsNum':docsNum, 'idf_all':idf_all})
        print(f'단어의 개수: {len(itb)}개')

        max_docsNum = max(itb['docsNum'])
        itb['idate'] = parse(idate)
        itb['df_sec'] = itb.docsNum / max_docsNum
        itb['i_value'] = i_value_cal(a, itb.df_sec, itb.idf_all)
        itb['rank_i'] = itb.i_value.rank(method='min', ascending=False).astype('int')

        itb.sort_values(['rank_i'], inplace=True, ascending=True)
        itb.reset_index(drop=True, inplace=True)
        itb = itb[['idate','term','docsNum','df_sec','idf_all','i_value','rank_i']]
        itb = itb.iloc[:100]
        itb[itb['term'] != '경수']
        print(f'{"-":->50}')
        kw_idate=pd.concat([kw_idate,itb.iloc[:5,:2]])
        itb_idate = pd.concat([itb_idate,itb])
    #     a_temp = itb.iloc[:11,1]
    #     a_table = pd.concat([a_table, a_temp], axis=1, join_axes=[a_table.index])
    # a_table
    kw_idate['section'] = section
    kw_total = pd.concat([kw_total,it_idate])
    itb_idate.to_sql(f'I_value_{section}', con=conn, if_exists='append')
    cur.execute(f'SELECT * FROM I_value_{section}').fetchall()
    conn.commit()

In [ ]:
try:
    cur.execute("CREATE TABLE I_value(index      INTEGER PRIMARY KEY, \
                                      idate      DATE, \
                                      term       TEXT, \
                                      section    TEXT, \
                                      article    REAL, \
                                      FOREIGN KEY (term) REFERENCES U_Term(u_term))")
    conn.commit()
except:
    pass